# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
%cd '/Users/krishansingh/Documents/algorithmic-trading-python-master/starter_files/'


/Users/krishansingh/Documents/algorithmic-trading-python-master/starter_files


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
IEX_CLOUD_API_TOKEN='Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 126941706, 'calculationPrice': 'close', 'change': 2.72, 'changePercent': 0.01563, 'close': 0, 'closeSource': 'ilfocafi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'eyidpeur a iec mt5ln1de', 'highTime': 1696182357813, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 175.9, 'iexCloseTime': 1700589229835, 'iexLastUpdated': 1688164861082, 'iexMarketPercent': 0.008427080564540458, 'iexOpen': 181.36, 'iexOpenTime': 1679722257671, 'iexRealtimePrice': 181.85, 'iexRealtimeSize': 6, 'iexVolume': 814016, 'lastTradeTime': 1714310783395, 'latestPrice': 177.19, 'latestSource': 'Close', 'latestTime': 'December 22, 2021', 'latestUpdate': 1678180298245, 'latestVolume': None, 'low': 0, 'lowSource': 'l eydai51ne emri cdtupe', 'lowTime': 1670393445884, 'ma

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
df = pd.DataFrame(columns=my_columns)
df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [8]:
df.append(
pd.Series([symbol, price,market_cap,'NA'], index=my_columns), ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,177.19,2883173921554,NA


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
df=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(
        pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A'], index=my_columns), 
        ignore_index=True)

In [11]:
df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,160.33,48632599488,N/A
1,AAL,18.93,11927501885,N/A
2,AAP,236.20,14891230381,N/A
3,AAPL,177.82,2965675211524,N/A
4,ABBV,134.83,239091311155,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [18]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [36]:
%timeit
symbol_groups=list(chunks(stocks['Ticker'], 100))
symbol_strings=[]
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
df=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'NA'], index=my_columns), ignore_index=True)

df


,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,160.25,47616644757,NA
1,AAL,19.17,12307859196,NA
2,AAP,233.95,14919472817,NA
3,AAPL,181.57,2989471956608,NA
4,ABBV,136.78,237618098780,NA
...,...,...,...,...
500,YUM,136.48,40487904740,NA
501,ZBH,129.86,26561316950,NA
502,ZBRA,596.99,31638731510,NA
503,ZION,63.74,9887490680,NA


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [43]:
portfolio_size=input("Enter the value of your portflio")

try:
    val=float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number\n")
    portfolio_size=input("Enter the value of your portflio")


Enter the value of your portflio10000000
10000000.0


In [44]:
position_size = val/len(df.index)
for i in range(0,len(df.index)):
    df.loc[i,'Number of shares to buy']=math.floor(position_size/ df.loc[i,'Stock Price'])

df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,160.25,47616644757,123
1,AAL,19.17,12307859196,1032
2,AAP,233.95,14919472817,84
3,AAPL,181.57,2989471956608,109
4,ABBV,136.78,237618098780,144
...,...,...,...,...
500,YUM,136.48,40487904740,145
501,ZBH,129.86,26561316950,152
502,ZBRA,596.99,31638731510,33
503,ZION,63.74,9887490680,310


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [68]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [69]:
background_color='#0a0a23'
font_color='#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color':background_color,
        'border':1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [70]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [73]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

## Saving Our Excel Output

Saving our Excel file is very easy:

In [74]:
 writer.save()